<a href="https://colab.research.google.com/github/RahmanKhorramfar91/UAI_AI_Energy_Case_Studies/blob/main/Case%20Study%203%20-%20Power%20Grid%20Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Case Study 3 - Universal AI, AI and Sustainability: Energy**

This case study is prepared by [Rahman Khorramfar](https://www.rahmankhorramfar.com/) for the ***AI and Sustainability: Energy*** module instructed by Professor [Saurabh Amin](https://cee.mit.edu/people_individual/saurabh-amin/). The module is part of the Universal AI course.

My email for questions, suggestions, and comments: khorram@mit.edu

## About the Case Study

This case study focuses on the planning and optimization of energy systems, which represent downstream applications leveraging the outputs of forecasting and outlier detection. Specifically, we delve into Economic Dispatch (ED) and Unit Commitment (UC) problems, widely employed by various entities to minimize electricity generation costs while ensuring system reliability and meeting demand. Efficient solutions to these problems ultimately enhance the social welfare of all stakeholders, including generators, regulators, and consumers.

We began with a short background and motivation for these problems to highlight their importance and how different market structures may approach each problem.
We then provide implementation for each model under deterministic, stochastic, and robust paradigms. It's important to note that these models and implementations are generalized frameworks, serving as foundations for more specialized models used in practice. While the typical planning horizons differ (5-15 minutes for ED, 1-3 hours with a day-to-week horizon for UC), for presentation convenience, we **solve them over a single day** on an hourly basis. For the ease of presentation, we present these model **without the network structure** that characterizes the real-world applications.
The case study concludes with questions that to add new constraints to the UC problem, and compare the out-of-sample performance for different modeling approaches.


**Overview of the content**:

1. [Background and Motivation](#background)
2. [Getting Started](#getting_started)
3. [Economic Dispatch](#ED)
4. [Unit Commitment](#UC)
5. [Value of Stochastic Solution (VSS)](#VSS)
8. [Case Study Questions](#questions)
9. [References and Further Reading](#references)

# **<font color='blue'>1. Background and Motivation </font>** [<a name="background"></a>](https://)

There are two primary structural approaches to managing the electricity supply chain and its market.

## **Vertically Integrated Utilities (VIU)**
Vertically integrated utilities are state or private companies that own and control the electricity supply chain, including generation, transmission, and distribution. This model leads to a monopoly regulated by the government to ensure fair prices and reliable service, thus offering benefits such as seamless coordination along the supply chain, lower financial risks, and predictable long-term planning. However, it comes with a slew of drawbacks including:

- Guaranteed customer base leads to **little incentive for innovation**
- No competition leads to inefficiencies and slower technology adoption, thus **higher prices** in the long run
- The monopoly leaves the customers with no choice, **exposed them to risk** associated with market volatility and price shifts.
- Managing a large supply chain by a single entity can lead to organizational complexity and higher bureaucracy, which can add **uncertainty** and make planning more difficult.

In response to these issues, the US Federal Energy Regulatory Commission (FERC) implemented a [series of reforms in the late 1990s](https://www.ferc.gov/power-sales-and-markets/rtos-and-isos#:~:text=Regional%20Transmission%20Organizations/Independent%20System,become%20a%20Regional%20Transmission%20Organization.) to:
- promote transparency in operations and price setting
- unbundle transmission from generation and distribution
- mandate fair use of transmission by all market participants
- encourage the formation of Independent System Operators (ISOs)

## **Independent System Operator (ISO)**
[ISOs](https://www.ferc.gov/electric-power-markets) are non-profit, independent entities that are regulated by FERC (except ERCOT in Texas), manage the wholesale market, and ensure grid reliability. They operate and facilitate non-discriminatory access to transmission lines which are typically owned by other entities
 Generally, the operations of ISOs involve managing a multi-layered market, monitoring grid operations, forecasting (load, renewable generation, and line congestion), and conducting long-term planning studies. The market management consists of the following layers:
- Real-Time (RT): given available generators, adjust generation to match the demand by solving **Economic Dispatch (ED)** problem every 5-15 minutes for the next few periods
- Day-Ahead (DA): schedule generators for each hour of the next day (or week) by solving **Unit Commitment (UC)** problem
- Ancillary Service  and Capacity Market (some ISOs): these are additional markets offered by some ISOs to secure reserves that support grid reliability and mitigate unexpected fluctuations

**Same Model, Different Mindsets**
While both VIU and ISO solve EC and UC with the same fundamental goals (schedule generators and meet demand at the lowest cost), they differ in the context and objectives. For example, VIU minimizes total production cost while ISO sets prices and maximizes the social welfare.

# **<font color='blue'>2. Getting Started </font>**  <a name="getting_started"></a>

Like previous case studies, we will work with the historical ISONE data available at [the GitHub repository](https://github.com/RahmanKhorramfar91/UAI_AI_Energy_Case_Studies).



In [1]:
import numpy as np; # for scientific computing
import pandas as pd; # for data science
import matplotlib.pyplot as plt; # for plotting
from statsmodels.tsa.seasonal import seasonal_decompose; # for statistical models including forecasting
from statsmodels.tsa.stattools import adfuller;
from statsmodels.tsa.arima.model import ARIMA;
import xgboost as xgb; # for XGBoos technique
from xgboost import plot_importance, plot_tree;
import os; # for directory control
import zipfile; # to handle zip files
import time;
import tensorflow as tf; # tensorflow
from tensorflow.keras.models import Sequential;
from tensorflow.keras.layers import *;
from tensorflow.keras.callbacks import ModelCheckpoint;
from tensorflow.keras.losses import MeanSquaredError;
from tensorflow.keras.metrics import RootMeanSquaredError;
from tensorflow.keras.optimizers import Adam;
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf; # for ACF and PACF plots used in ARIMA
np.random.seed(2025);

Make sure to install the [`pyoptinterface` package](https://metab0t.github.io/PyOptInterface/index.html), which is an open-source Python library that provides a unified and [high-performance](https://metab0t.github.io/PyOptInterface/benchmark.html) interface. Similar to Pyomo in Python and JuMP in Julia, this package makes switching between solvers much easier.

In [2]:
# install and fetch the pyoptinterface, an open-source Python library that provides a unified, high-performance interface
!pip install pyoptinterface[highs]
import pyoptinterface as poi;
from pyoptinterface import highs;

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.6 MB/s eta 0:00:00


Clone the repo, summon the data for the intended year, and add some temporal features.

In [3]:
!git clone 'https://github.com/RahmanKhorramfar91/UAI_AI_Energy_Case_Studies.git';
zip_file_path = os.getcwd()+'/UAI_AI_Energy_Case_Studies/load_time_series_data_ISONE_2000-2023.zip';
extract_dir = os.getcwd()+'/load_time_series_data_ISONE_2000-2023';
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir);


Cloning into 'UAI_AI_Energy_Case_Studies'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 76 (delta 42), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 9.61 MiB | 11.85 MiB/s, done.
Resolving deltas: 100% (42/42), done.


Here, we assume each of the historical records as a possible realization. We randomly sample $nS$ scenarios out of 24 to form our scenario set. Therefore, we assume that the demand/supply parameters in our model is going to take one of its historical values. In practice, however, we may want to go beyong historical records and include synthetic scenarios.

We also standardize the irradiance and wind speed to approximate the solar capacity factor.

In [4]:
np.random.seed(2025);
nS = 5;
selected_day = 200;
Scens = np.random.randint(2000, 2024, size=nS);
demand = np.zeros((nS, 24));
solar_CF = np.zeros((nS, 24));
wind_CF = np.zeros((nS, 24));
for si, sv in enumerate(Scens):
  data = pd.read_csv(f'/content/load_time_series_data_ISONE_2000-2023/load_time_series_data/{sv}_ISONE_load.csv');
  data.index = pd.to_datetime(data.date);
  data['date'] = data.index;
  data['day_of_year'] = data['date'].dt.dayofyear;
  demand[si,:] = data['gross_load_MW'][data['day_of_year']==selected_day].to_numpy();
  solar_CF[si,:] = data['global_horizontal_irradiance'][data['day_of_year']==selected_day].to_numpy();
  solar_CF[si,:] = solar_CF[si,:]/solar_CF[si,:].max();
  wind_CF[si,:] = data['10m_wind_speed'][data['day_of_year']==selected_day].to_numpy();
  wind_CF[si,:] = wind_CF[si,:]/wind_CF[si,:].max();

print(solar_CF[0,:10]);

[0.         0.         0.         0.         0.         0.01857065
 0.13524333 0.32682143 0.5299228  0.7104504 ]


# **<font color='blue'>3. Economic Dispatch </font>**  <a name="ED"></a>

ED determines the power generation from each committed generator to meet the electricity demand at the lowest operating cost under power system constraints.



## Simplest Form

**Sets**
- $\mathcal{G}$: generators, indexed by $g$
- $\mathcal{T}$: planning hours, indexed by $t$

**Parameters**
- $c_g$: Marginal generation cost [\$/MWh]
- $c^{\text{VoLL}}$: value of lost load [\$/MWh]
- $P^{\text{min}}_g$: minimum stable generation, [MW]
- $P^{\text{max}}_g$: nameplate capacity, [MW]
- $R^{\text{up}}_g, R^{\text{dn}}_g$: ramp-up/down rates [MW/hour]
- $D_t$: system demand [MW]       

**Variables**
- $p_{gt}\in \mathbb{R}^+$: power generation [MW]
- $a_t \in \mathbb{R}^+$: lost load [MW]

**<font color='blue'>Mathematical Model </font>**   

\begin{align*}
    \min & \sum_{g\in \mathcal{G}} \sum_{t\in \mathcal{T}} c_g p_{gt} + \sum_{t\in \mathcal{T}}c^{\text{VoLL}} a_t \\
    \text{s.t.} \ & \sum_{g\in \mathcal{G}} p_{gt} + a_t = D_t & t\in \mathcal{T}\\
    & P^{\text{min}}_g \leq p_{gt} \leq P^{\text{max}}_g & g\in \mathcal{G}, t\in \mathcal{T}\\
    & p_{gt}-p_{g,t-1} \leq R^{\text{up}}_g P^{\text{max}}_g & g\in \mathcal{G}, t\in \mathcal{T}\backslash t_0\\
    & p_{g,t-1}-p_{gt} \leq R^{\text{dn}}_g P^{\text{max}}_g & g\in \mathcal{G}, t\in \mathcal{T}\backslash t_0
\end{align*}

**<font color='blue'>Note </font>**: We mention that the ISO solves ED to maximize the welfare. However, the model only minimizes the production cost. So why is the discrepancy? The answer lies in the original definition of the objective function, which is:

$max \ U(D)-C(p)$

where $U(D)$ is the utility generated by consuming demand and $C(p)$ is the production cost. Since the demand is assumed to be almost inelastic (i.e., increasing the price does not significantly change the consumption) in most situations, the first term simply becomes a constant and can be dropped. In other words, although both market structures seemingly solve similar problems, ISOs solve it as a *market-clearing* problem to determine the price at an equilibrium (more on Section 7 resources).




The implementation is given below for the first scenario demand and capacity factors. Note that the naming convention in the coding might differ from its mathematical formulation, and that many values of many parameters are generated randomly which may not resemble their actual quantities in practice.

In [7]:
np.random.seed(2025);
time_stamp = time.time();
### SETS
nGen = 15;
nT = len(demand[0]);

### PARAMETERS
gen_cost = np.random.randint(low=2, high=5, size=nGen); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.5*demand[0].max()/nGen), int(1.5*demand[0].max()/nGen), size=nGen);
Pmin = np.zeros(nGen);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nGen);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nGen);

# MODEL INSTANCE
Model = highs.Model();

### DECISION VARIABLES
# define a class to distinguish the variables from parameters
class DV(): prod=[]; LL=[]; obj=[];

DV.prod = Model.add_variables(range(nGen), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production, continuous positive variable
DV.LL = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
DV.obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function, free variable

### OBJECTIVE FUNCTION
obj = poi.ExprBuilder();
obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod[g,t] for g in range(nGen)) + LL_cost*DV.LL[t] for t in range(nT));
Model.add_linear_constraint(DV.obj-obj, poi.Eq, 0);
Model.set_objective(DV.obj, poi.ObjectiveSense.Minimize)

### CONSTRAINTS
# balance equation
for t in range(nT):
  Model.add_linear_constraint(poi.quicksum(DV.prod[g,t] for g in range(nGen))+DV.LL[t], poi.Eq, demand[0,t]);

# production capacity
for g in range(nGen):
  for t in range(nT):
    Model.add_linear_constraint(DV.prod[g,t], poi.Leq, Pmax[g]);
    Model.add_linear_constraint(DV.prod[g,t], poi.Geq, Pmin[g]);

# ramping
for g in range(nGen):
  for t in range(1, nT):
    Model.add_linear_constraint(DV.prod[g,t]-DV.prod[g,t-1], poi.Leq, RampU[g]*Pmax[g]);
    Model.add_linear_constraint(DV.prod[g,t-1]-DV.prod[g,t], poi.Leq, RampD[g]*Pmax[g]);

### SOLVE THE MODEL
Model.optimize();

### POST-PROCESSING
# calculate total generation and load shedding
total_gen, lost_load = 0, 0;
for t in range(nT):
    for g in range(nGen): total_gen += Model.get_value(DV.prod[g,t]);
    lost_load += Model.get_value(DV.LL[t]);

print(f'Objective value: {np.round(Model.get_value(obj),1)}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');
print(f'Generation: {round(total_gen)} MWh, \t lost load: {round(lost_load)} MWh,\t total load: {round(demand[0].sum())}');

Objective value: 1027362.5, 	 Elapsed time: 0.04 (s)
Generation: 358793 MWh, 	 lost load: 0 MWh,	 total load: 358793


<br><br>

## ED with Variable Renewables Energy (VRE) and Storage

**New Sets**
- $\mathcal{H}$: thermal generators, indexed by $g$
- $\mathcal{V}$: variable renewables, indexed by $g$
- $\mathcal{G} = \mathcal{H} \cup \mathcal{V}$: all generators

**New Variables**

- $s^{\text{soc}}_t \in \mathbb{R}^+$: storage state of charge (level), [MWh]
- $s^{\text{ch}}_t, s^{\text{dis}}_t \in \mathbb{R}^+$: storage charge/discharge, [MW]

**New Parameters**
- $\rho_{gt}$: capacity factor for renewables [MW]
- $y^{\text{lev}}, y^{\text{ch}},y^{\text{dis}}$: storage energy and charge/discharge capacity, [MWh] and [MW]
- $\gamma^{\text{ch}}, \gamma^{\text{dis}}$: charge/discharge efficiency       

**<font color='blue'>Mathematical Model </font>**   
    \begin{align}
    \min & \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}} c_g p_{gt} + \sum_{t\in \mathcal{T}}c^{\text{VoLL}} a_t\\
    \text{s.t.} \ & \sum_{g\in \mathcal{G}} p_{gt} + s^{\text{dis}}_t - s^{\text{ch}}_t+a_t = D_t & t\in \mathcal{T}\\
    & P^{\text{min}}_g \leq p_{gt} \leq P^{\text{max}}_g & g\in \mathcal{G}, t\in \mathcal{T}\\
    & p_{gt}-p_{g,t-1} \leq R^{\text{up}}_g P^{\text{max}}_g, \qquad  p_{g,t-1}-p_{gt} \leq R^{\text{dn}}_g P^{\text{max}}_g & g\in \mathcal{H}, t\in \mathcal{T}\backslash t_0\\
    & p_{gt}\leq \rho_{gt}P^{\text{max}}_g & g\in \mathcal{V}, t\in \mathcal{T}\\
    &s^{\text{soc}}_t = s^{\text{soc}}_{t-1}+ \gamma^{\text{ch}} s^{\text{ch}}_t - \frac{s^{\text{dis}}_t}{\gamma^{\text{dis}}}, \qquad s^{\text{soc}}_0=0& t\in \mathcal{T}\backslash t_0\\
    &s^{\text{soc}}_t\leq y^{\text{lev}}, \quad s^{\text{ch}}_t\leq y^{\text{ch}} ,\quad s^{\text{ch}}_t\leq  y^{\text{dis}}& t\in \mathcal{T}
\end{align}


In [8]:
np.random.seed(2025);
time_stamp = time.time();
### SETS
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
gen_cost = np.random.randint(low=2, high=5, size=nGen); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nGen);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nGen);
store_level = 0.2*demand[0].max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;

# MODEL INSTANCE
Model = highs.Model();


### DECISION VARIABLES
# define a class to distinguish the variables from parameters
class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[]; obj=[];

DV.prod_thermal = Model.add_variables(range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production from thermal generators, continuous positive variable
DV.prod_sol = Model.add_variables(range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production from solar, continuous positive variable
DV.prod_wind = Model.add_variables(range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production from wind, continuous positive variable
DV.LL = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
DV.SOC = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
DV.store_ch = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
DV.store_dis = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
DV.obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function, free variable

### OBJECTIVE FUNCTION
obj = poi.ExprBuilder();
obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[g,t] for g in range(nThermal)) + LL_cost*DV.LL[t] for t in range(nT));
Model.add_linear_constraint(DV.obj-obj, poi.Eq, 0);
Model.set_objective(DV.obj, poi.ObjectiveSense.Minimize)

### CONSTRAINTS
# balance equation
for t in range(nT):
  Model.add_linear_constraint(
      poi.quicksum(DV.prod_thermal[g,t] for g in range(nThermal))+  # generation from thermal plants
      poi.quicksum(DV.prod_sol[g,t] for g in range(nSolar))+        # generation from solar
      poi.quicksum(DV.prod_wind[g,t] for g in range(nWind))+        # generation from wind
      DV.store_dis[t]-DV.store_ch[t]+DV.LL[t],                      # discharge-charge + LL
      poi.Eq, demand[0, t]);

# production capacity for thermal plants
for g in range(nThermal):
  for t in range(nT):
    Model.add_linear_constraint(DV.prod_thermal[g,t], poi.Leq, Pmax[g]);
    Model.add_linear_constraint(DV.prod_thermal[g,t], poi.Geq, Pmin[g]);

# ramping
for g in range(nThermal):
  for t in range(1, nT):
    Model.add_linear_constraint(DV.prod_thermal[g,t]-DV.prod_thermal[g,t-1], poi.Leq, RampU[g]*Pmax[g]);
    Model.add_linear_constraint(DV.prod_thermal[g,t-1]-DV.prod_thermal[g,t], poi.Leq, RampD[g]*Pmax[g]);

# availability for VRE
for t in range(nT):
  for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[g,t], poi.Leq, Pmax[g+nThermal]*solar_CF[0, t]);
  for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[g,t], poi.Leq, Pmax[g+nThermal+nSolar]*wind_CF[0, t]);

# storage
for t in range(1, nT):
  Model.add_linear_constraint(DV.SOC[t]-DV.SOC[t-1]-store_eff_ch*DV.store_ch[t]+DV.store_dis[t]/store_eff_dis, poi.Eq, 0);

Model.add_linear_constraint(DV.SOC[0], poi.Eq,0);
for t in range(nT):
  Model.add_linear_constraint(DV.SOC[t], poi.Leq, store_level);
  Model.add_linear_constraint(DV.store_ch[t], poi.Leq, store_ch);
  Model.add_linear_constraint(DV.store_dis[t], poi.Leq, store_dis);

### SOLVE THE MODEL
Model.optimize();

### POST-PROCESSING
# calculate total generation and load shedding
total_thermal_gen, lost_load = 0, 0;
total_VRE_gen = 0;
for t in range(nT):
    for g in range(nThermal): total_thermal_gen += Model.get_value(DV.prod_thermal[g,t]);
    for g in range(nSolar): total_VRE_gen += Model.get_value(DV.prod_sol[g,t]);
    for g in range(nWind): total_VRE_gen += Model.get_value(DV.prod_wind[g,t]);
    lost_load += Model.get_value(DV.LL[t]);

print(f'Objective value: {np.round(Model.get_value(obj),1)}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');
print(f'Thermal gen: {round(total_thermal_gen)} MWh,  VRE gen: {round(total_VRE_gen)} MWh,\
 \t lost load: {round(lost_load)} MWh,\t total load: {round(demand[0].sum())}');

Objective value: 428794.3, 	 Elapsed time: 0.03 (s)
Thermal gen: 188910 MWh,  VRE gen: 169739 MWh, 	 lost load: 0 MWh,	 total load: 358793


<br><br>

## Stochastic ED

Several parameters and variables are adjusted with a subscript $s$ to indicate their value at scenario $s$


**New Sets**
- $\mathcal{S}$: Possible realization scenarios (discrete set)



**New Parameters**
- $\pi_s$: scenario probability

**<font color='blue'>Mathematical Model </font>**   
    \begin{align}
    \min & \sum_{s\in \mathcal{S}}\pi_s [ \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}} c_g p_{sgt} + \sum_{t\in \mathcal{T}}c^{\text{VoLL}} a_{st}]\\
    \text{s.t.} \ & \sum_{g\in \mathcal{G}} p_{sgt} + s^{\text{dis}}_{st} - s^{\text{ch}}_{st}+a_{st} = D_{st} & s\in \mathcal{S}, t\in \mathcal{T}\\
    & P^{\text{min}}_g \leq p_{sgt} \leq P^{\text{max}}_g & s\in \mathcal{S}, g\in \mathcal{G}, t\in \mathcal{T}\\
    & p_{sgt}-p_{sg,t-1} \leq R^{\text{up}}_g P^{\text{max}}_g, \qquad  p_{sg,t-1}-p_{sgt} \leq R^{\text{dn}}_g P^{\text{max}}_g & s\in \mathcal{S}, g\in \mathcal{H}, t\in \mathcal{T}\backslash t_0\\
    & p_{sgt}\leq \rho_{sgt}P^{\text{max}}_g & s\in \mathcal{S}, g\in \mathcal{V}, t\in \mathcal{T}\\
    &s^{\text{soc}}_{st} = s^{\text{soc}}_{s,t-1}+ \gamma^{\text{ch}} s^{\text{ch}}_{st} - \frac{s^{\text{dis}}_{st}}{\gamma^{\text{dis}}}, \qquad s^{\text{soc}}_{s0}=0&s\in \mathcal{S}, t\in \mathcal{T}\backslash t_0\\
    &s^{\text{soc}}_{st}\leq y^{\text{lev}}, \quad s^{\text{ch}}_{st}\leq y^{\text{ch}} ,\quad s^{\text{ch}}_{st}\leq  y^{\text{dis}}&s\in \mathcal{S}, t\in \mathcal{T}
\end{align}

**Note that the stochastic ED does not have a here-an-now (stage 1) decisions.**

Randomly select $|\mathcal{S} |$ days, each representing a discrete and independent scenario of demands and capacity factors. Note that we make the independent assumptions for the ease of implementation. In practice, each scenario may represent a possible realization for the same time period.


In [11]:
np.random.seed(2025);
time_stamp = time.time();
### SETS
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
gen_cost = np.random.randint(low=2, high=5, size=nThermal); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nThermal);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nThermal);
store_level = 0.2*demand.max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;
probs = np.random.dirichlet(np.ones(nS), size=1)[0];

# MODEL INSTANCE
Model = highs.Model();


### DECISION VARIABLES
# define a class to distinguish the variables from parameters
class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[]; objs=[];

DV.prod_thermal = Model.add_variables(range(nS), range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production from thermal generators, continuous positive variable
DV.prod_sol = Model.add_variables(range(nS), range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production from solar, continuous positive variable
DV.prod_wind = Model.add_variables(range(nS), range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous); #production from wind, continuous positive variable
DV.LL = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
DV.SOC = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
DV.store_ch = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
DV.store_dis = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
DV.objs = Model.add_variables(range(nS), domain=poi.VariableDomain.Continuous); # objective function, free variable

### OBJECTIVE FUNCTION
for s in range(nS):
  obj = poi.ExprBuilder();
  obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[s,g,t] for g in range(nThermal)) + LL_cost*DV.LL[s, t] for t in range(nT));
  Model.add_linear_constraint(DV.objs[s]-obj, poi.Eq, 0);
Model.set_objective(poi.quicksum(probs[s]*DV.objs[s] for s in range(nS)), poi.ObjectiveSense.Minimize)

### CONSTRAINTS
for s in range(nS):
  # balance equation
  for t in range(nT):
    Model.add_linear_constraint(
        poi.quicksum(DV.prod_thermal[s,g,t] for g in range(nThermal))+  # generation from thermal plants
        poi.quicksum(DV.prod_sol[s,g,t] for g in range(nSolar))+        # generation from solar
        poi.quicksum(DV.prod_wind[s,g,t] for g in range(nWind))+        # generation from wind
        DV.store_dis[s,t]-DV.store_ch[s,t]+DV.LL[s,t],                      # discharge-charge + LL
        poi.Eq, demand[s,t]);

  # production capacity for thermal plants
  for g in range(nThermal):
    for t in range(nT):
      Model.add_linear_constraint(DV.prod_thermal[s,g,t], poi.Leq, Pmax[g]);
      Model.add_linear_constraint(DV.prod_thermal[s,g,t], poi.Geq, Pmin[g]);

  # ramping
  for g in range(nThermal):
    for t in range(1, nT):
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-DV.prod_thermal[s,g,t-1], poi.Leq, RampU[g]*Pmax[g]);
      Model.add_linear_constraint(DV.prod_thermal[s,g,t-1]-DV.prod_thermal[s,g,t], poi.Leq, RampD[g]*Pmax[g]);

  # availability for VRE
  for t in range(nT):
    for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[s,g,t], poi.Leq, Pmax[g+nThermal]*solar_CF[s,t]);
    for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[s,g,t], poi.Leq, Pmax[g+nThermal+nSolar]*wind_CF[s,t]);

  # storage
  for t in range(1, nT):
    Model.add_linear_constraint(DV.SOC[s,t]-DV.SOC[s,t-1]-store_eff_ch*DV.store_ch[s,t]+DV.store_dis[s,t]/store_eff_dis, poi.Eq, 0);

  Model.add_linear_constraint(DV.SOC[s,0], poi.Eq,0);
  for t in range(nT):
    Model.add_linear_constraint(DV.SOC[s,t], poi.Leq, store_level);
    Model.add_linear_constraint(DV.store_ch[s,t], poi.Leq, store_ch);
    Model.add_linear_constraint(DV.store_dis[s,t], poi.Leq, store_dis);

### SOLVE THE MODEL
Model.optimize();

### POST-PROCESSING
# calculate total generation and load shedding
total_thermal_gen, lost_load = np.zeros(nS), np.zeros(nS);
total_VRE_gen = np.zeros(nS);
for s in range(nS):
  for t in range(nT):
      for g in range(nThermal): total_thermal_gen[s] += Model.get_value(DV.prod_thermal[s,g,t]);
      for g in range(nSolar): total_VRE_gen[s] += Model.get_value(DV.prod_sol[s,g,t]);
      for g in range(nWind): total_VRE_gen[s] += Model.get_value(DV.prod_wind[s,g,t]);
      lost_load[s] += Model.get_value(DV.LL[s,t]);
scenario_obj = np.zeros(nS);
for s in range(nS):
  scenario_obj[s] = round(Model.get_value(DV.objs[s]));
  print(f'scenario {s}, prob: {np.round(probs[s],2)}, obj = {scenario_obj[s]}, Thermal gen: {round(total_thermal_gen[s])} MWh,  VRE gen: {round(total_VRE_gen[s])} MWh,\
   lost load: {round(lost_load[s])} MWh,\t total load: {round(demand[s].sum())}');
print(f'\n Total Objective value: {np.round(Model.get_model_attribute(poi.ModelAttribute.ObjectiveValue),1)}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');

# # print(Model.get_model_attribute(poi.ModelAttribute.TerminationStatus));

scenario 0, prob: 0.08, obj = 397773.0, Thermal gen: 177816 MWh,  VRE gen: 180778 MWh,   lost load: 0 MWh,	 total load: 358793
scenario 1, prob: 0.18, obj = 823814.0, Thermal gen: 300525 MWh,  VRE gen: 190740 MWh,   lost load: 0 MWh,	 total load: 492422
scenario 2, prob: 0.33, obj = 266422.0, Thermal gen: 132701 MWh,  VRE gen: 213348 MWh,   lost load: 0 MWh,	 total load: 346272
scenario 3, prob: 0.06, obj = 535842.0, Thermal gen: 228019 MWh,  VRE gen: 194999 MWh,   lost load: 0 MWh,	 total load: 423425
scenario 4, prob: 0.34, obj = 823814.0, Thermal gen: 300525 MWh,  VRE gen: 190740 MWh,   lost load: 0 MWh,	 total load: 492422

 Total Objective value: 587100.7, 	 Elapsed time: 0.11 (s)


<br><br>

# **<font color='blue'>4. Unit Commitment </font>**  <a name="UC"></a>

UC determines the commitment status (whether it is on or not) for generators over a time period of usually a day or a week. The objective is to reduce the cost of commitment and operation, subject to the system constraints.

## Deterministic UC

**New Parameters**
- $c^{\text{cm}}_g$: commitment cost
- $c^{\text{su}}_g$: startup cost
- $c^{\text{sd}}_g$: shutdown cost


**New Variables**
- $\theta_s$: objective value of dispatch problem
- $u^{\text{cm}}_{gt}\in \{0, 1\}$: generator on/off status
- $u^{\text{su}}_{gt}, u^{\text{sd}}_{gt}\in \{0, 1\}$: startup and shutdown decisions

**<font color='blue'>Mathematical Model </font>**   
    \begin{align}
    \min & \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}}(c^{\text{cm}}_g u^{\text{cm}}_{gt}+ c^{\text{su}}_g u^{\text{su}}_{gt}+c^{\text{sd}}_g u^{\text{sd}}_{gt})+  \theta\\
    \text{s.t.} \ & \theta =  \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}} c_g p_{gt} + \sum_{t\in \mathcal{T}}c^{\text{VoLL}} a_{t} & \\
    &u^{\text{cm}}_{gt} = u^{\text{cm}}_{g,t-1}+u^{\text{su}}_{gt}-u^{\text{sd}}_{gt}& g\in \mathcal{G}, t\in \mathcal{T}\\
    & \sum_{g\in \mathcal{G}} p_{gt} + s^{\text{dis}}_{t} - s^{\text{ch}}_{t}+a_{t} = D_{t} & t\in \mathcal{T}\\
    & P^{\text{min}}_g u^{\text{cm}}_{gt} \leq p_{gt} \leq P^{\text{max}}_g u^{\text{cm}}_{gt} & g\in \mathcal{G}, t\in \mathcal{T}\\
    & p_{gt}-p_{g,t-1} \leq R^{\text{up}}_g P^{\text{max}}_gu^{\text{cm}}_{gt}, \qquad  p_{g,t-1}-p_{gt} \leq R^{\text{dn}}_g P^{\text{max}}_g u^{\text{cm}}_{gt} & g\in \mathcal{H}, t\in \mathcal{T}\backslash t_0\\
    & p_{gt}\leq \rho_{gt}P^{\text{max}}_g & g\in \mathcal{V}, t\in \mathcal{T}\\
    &s^{\text{soc}}_{t} = s^{\text{soc}}_{t-1}+ \gamma^{\text{ch}} s^{\text{ch}}_{t} - \frac{s^{\text{dis}}_{t}}{\gamma^{\text{dis}}}, \qquad s^{\text{soc}}_{0}=0&t\in \mathcal{T}\backslash t_0\\
    &s^{\text{soc}}_{t}\leq y^{\text{lev}}, \quad s^{\text{ch}}_{t}\leq y^{\text{ch}} ,\quad s^{\text{ch}}_{t}\leq  y^{\text{dis}}&t\in \mathcal{T}
\end{align}


Consider a few scenarios, and independently solve the deterministic UC for each scenario. We will use these results to compare with stochastic and robust optimization paradigms.

In [12]:
UC_obj_det = np.zeros(nS);
np.random.seed(2025);

### Sets
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
# demand is the demand matrix, solar_CF and wind_CF are the solar and wind capacity factors
gen_cost = np.random.randint(low=2, high=7, size=nThermal); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nThermal);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nThermal);
store_level = 0.2*demand.max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;
commit_cost = np.random.randint(500, 1500, size=nThermal);
startup_cost = np.random.randint(1000, 3000, size=nThermal);
shutdown_cost = np.random.randint(200, 2000, size=nThermal);
probs = np.random.dirichlet(np.ones(nS), size=1)[0];

for s in range(nS):
  time_stamp = time.time();

  # MODEL INSTANCE
  Model = highs.Model();

  ### DECISION VARIABLES
  # define a class to distinguish the variables from parameters
  class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[];Ucom=[]; Usu=[]; Usd=[];s1_obj=[]; s2_obj=[];

  DV.prod_thermal = Model.add_variables( range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
  DV.prod_sol = Model.add_variables(range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
  DV.prod_wind = Model.add_variables(range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
  DV.LL = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
  DV.SOC = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
  DV.store_ch = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
  DV.store_dis = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
  DV.Ucom = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # generator on/off status
  DV.Usu = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # startup decision
  DV.Usd = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # shutdown decision
  DV.s1_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for first-stage problme, free variable
  DV.s2_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for second-stage problme, free variable

  ### OBJECTIVE FUNCTION
  obj_total = poi.ExprBuilder();
  # first-stage objective
  obj_total += poi.quicksum(poi.quicksum(commit_cost[g]*DV.Ucom[g,t]+ startup_cost[g]*DV.Usu[g,t]+shutdown_cost[g]*DV.Usd[g,t] for g in range(nThermal)) for t in range(nT));
  Model.add_linear_constraint(DV.s1_obj-obj_total, poi.Eq, 0);
  obj_total += DV.s2_obj;
  Model.set_objective(obj_total, poi.ObjectiveSense.Minimize);

  ### CONSTRAINTS
  # cost components
  obj = poi.ExprBuilder();
  obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[g,t] for g in range(nThermal)) + LL_cost*DV.LL[t] for t in range(nT));
  Model.add_linear_constraint(DV.s2_obj-obj, poi.Eq, 0);

  # on/off status
  for g in range(nThermal):
    for t in range(1, nT):
      Model.add_linear_constraint(DV.Ucom[g,t]-DV.Ucom[g,t-1]-DV.Usu[g,t]+DV.Usd[g,t], poi.Eq, 0);

  # balance equation
  for t in range(nT):
    Model.add_linear_constraint(
        poi.quicksum(DV.prod_thermal[g,t] for g in range(nThermal))+  # generation from thermal plants
        poi.quicksum(DV.prod_sol[g,t] for g in range(nSolar))+        # generation from solar
        poi.quicksum(DV.prod_wind[g,t] for g in range(nWind))+        # generation from wind
        DV.store_dis[t]-DV.store_ch[t]+DV.LL[t],                      # discharge-charge + LL
        poi.Eq, demand[s,t]);

  # production capacity for thermal plants
  for g in range(nThermal):
    for t in range(nT):
      Model.add_linear_constraint(DV.prod_thermal[g,t]-Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[g,t]-Pmin[g]*DV.Ucom[g,t], poi.Geq,0);

  # ramping
  for g in range(nThermal):
    for t in range(1, nT):
      Model.add_linear_constraint(DV.prod_thermal[g,t]-DV.prod_thermal[g,t-1] -RampU[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[g,t-1]-DV.prod_thermal[g,t] -RampD[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);

  # availability for VRE
  for t in range(nT):
    for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[g,t], poi.Leq, Pmax[g+nThermal]*solar_CF[s,t]);
    for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[g,t], poi.Leq, Pmax[g+nThermal+nSolar]*wind_CF[s,t]);

  # storage
  for t in range(1, nT):
    Model.add_linear_constraint(DV.SOC[t]-DV.SOC[t-1]-store_eff_ch*DV.store_ch[t]+DV.store_dis[t]/store_eff_dis, poi.Eq, 0);

  Model.add_linear_constraint(DV.SOC[0], poi.Eq,0);
  for t in range(nT):
    Model.add_linear_constraint(DV.SOC[t], poi.Leq, store_level);
    Model.add_linear_constraint(DV.store_ch[t], poi.Leq, store_ch);
    Model.add_linear_constraint(DV.store_dis[t], poi.Leq, store_dis);

  ### SOLVE THE MODEL
  Model.set_model_attribute(poi.ModelAttribute.Silent, 0);
  Model.optimize();

  ### POST-PROCESSING
  ll=0;
  for t in range(nT):
    ll += Model.get_value(DV.LL[t]);
  print(f' Scenario: {s}, first-stage obj: {round(Model.get_value(DV.s1_obj))}\
  second-stage obj: {round(Model.get_value(DV.s2_obj))}, lost load: {round(ll)}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');
  UC_obj_det[s] = Model.get_value(DV.s1_obj)+Model.get_value(DV.s2_obj);

print(np.round(UC_obj_det));

 Scenario: 0, first-stage obj: 128096  second-stage obj: 582020, lost load: 0, 	 Elapsed time: 2.08 (s)
 Scenario: 1, first-stage obj: 162186  second-stage obj: 1154318, lost load: 0, 	 Elapsed time: 1.2 (s)
 Scenario: 2, first-stage obj: 103656  second-stage obj: 396654, lost load: 0, 	 Elapsed time: 1.45 (s)
 Scenario: 3, first-stage obj: 138240  second-stage obj: 787414, lost load: 0, 	 Elapsed time: 0.83 (s)
 Scenario: 4, first-stage obj: 162186  second-stage obj: 1154318, lost load: 0, 	 Elapsed time: 1.2 (s)
[ 710116. 1316504.  500310.  925654. 1316504.]


<br><br>

## Stochastic UC

**New Parameters**
- $c^{\text{cm}}_g$: commitment cost
- $c^{\text{su}}_g$: startup cost
- $c^{\text{sd}}_g$: shutdown cost


**New Variables**
- $\theta_s$: objective value of dispatch problem
- $u^{\text{cm}}_{gt}\in \{0, 1\}$: generator on/off status
- $u^{\text{su}}_{gt}, u^{\text{sd}}_{gt}\in \{0, 1\}$: startup and shutdown decisions

**<font color='blue'>Mathematical Model </font>**   
    \begin{align}
    \min & \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}}(c^{\text{cm}}_g u^{\text{cm}}_{gt}+ c^{\text{su}}_g u^{\text{su}}_{gt}+c^{\text{sd}}_g u^{\text{sd}}_{gt})+ \sum_{s\in \mathcal{S}}\pi_s \theta_s\\
    \text{s.t.} \ & \theta_s =  \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}} c_g p_{sgt} + \sum_{t\in \mathcal{T}}c^{\text{VoLL}} a_{st} & s\in \mathcal{S}\\
    &u^{\text{cm}}_{gt} = u^{\text{cm}}_{g,t-1}+u^{\text{su}}_{gt}-u^{\text{sd}}_{gt}& g\in \mathcal{G}, t\in \mathcal{T}\\
    & \sum_{g\in \mathcal{G}} p_{sgt} + s^{\text{dis}}_{st} - s^{\text{ch}}_{st}+a_{st} = D_{st} & s\in \mathcal{S}, t\in \mathcal{T}\\
    & P^{\text{min}}_g u^{\text{cm}}_{gt} \leq p_{sgt} \leq P^{\text{max}}_g u^{\text{cm}}_{gt} & s\in \mathcal{S}, g\in \mathcal{G}, t\in \mathcal{T}\\
    & p_{sgt}-p_{sg,t-1} \leq R^{\text{up}}_g P^{\text{max}}_gu^{\text{cm}}_{gt}, \qquad  p_{sg,t-1}-p_{sgt} \leq R^{\text{dn}}_g P^{\text{max}}_g u^{\text{cm}}_{gt} & s\in \mathcal{S}, g\in \mathcal{H}, t\in \mathcal{T}\backslash t_0\\
    & p_{sgt}\leq \rho_{sgt}P^{\text{max}}_g & s\in \mathcal{S}, g\in \mathcal{V}, t\in \mathcal{T}\\
    &s^{\text{soc}}_{st} = s^{\text{soc}}_{s,t-1}+ \gamma^{\text{ch}} s^{\text{ch}}_{st} - \frac{s^{\text{dis}}_{st}}{\gamma^{\text{dis}}}, \qquad s^{\text{soc}}_{s0}=0&s\in \mathcal{S}, t\in \mathcal{T}\backslash t_0\\
    &s^{\text{soc}}_{st}\leq y^{\text{lev}}, \quad s^{\text{ch}}_{st}\leq y^{\text{ch}} ,\quad s^{\text{ch}}_{st}\leq  y^{\text{dis}}&s\in \mathcal{S}, t\in \mathcal{T}
\end{align}


In [13]:
np.random.seed(2025);
time_stamp = time.time();
### SETS
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
# demand is the demand matrix, solar_CF and wind_CF are the solar and wind capacity factors
gen_cost = np.random.randint(low=2, high=7, size=nThermal); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nThermal);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nThermal);
store_level = 0.2*demand.max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;
commit_cost = np.random.randint(500, 1500, size=nThermal);
startup_cost = np.random.randint(1000, 3000, size=nThermal);
shutdown_cost = np.random.randint(200, 2000, size=nThermal);
probs = np.random.dirichlet(np.ones(nS), size=1)[0];

# MODEL INSTANCE
Model = highs.Model();

### DECISION VARIABLES
# define a class to distinguish the variables from parameters
class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[]; Ucom=[]; Usu=[]; Usd=[]; theta=[];s1_obj=[]; total_obj=[]

DV.prod_thermal = Model.add_variables(range(nS), range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.prod_sol = Model.add_variables(range(nS), range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.prod_wind = Model.add_variables(range(nS), range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.LL = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
DV.SOC = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
DV.store_ch = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
DV.store_dis = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
DV.Ucom = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # generator on/off status
DV.Usu = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # startup decision
DV.Usd = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # shutdown decision
DV.theta = Model.add_variables(range(nS), domain=poi.VariableDomain.Continuous); # objective function for second-stage problmes, free variable
DV.s1_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for first-stage problme, free variable
DV.total_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # total objective function, free variable

### OBJECTIVE FUNCTION
obj_total = poi.ExprBuilder();
# first-stage objective
obj_total += poi.quicksum(poi.quicksum(commit_cost[g]*DV.Ucom[g,t]+ startup_cost[g]*DV.Usu[g,t]+shutdown_cost[g]*DV.Usd[g,t] for g in range(nThermal)) for t in range(nT));
Model.add_linear_constraint(DV.s1_obj-obj_total, poi.Eq, 0);
obj_total += poi.quicksum(probs[s]*DV.theta[s] for s in range(nS));
Model.add_linear_constraint(DV.total_obj-obj_total, poi.Eq, 0);
Model.set_objective(obj_total, poi.ObjectiveSense.Minimize)

### CONSTRAINTS
# cost components
for s in range(nS):
  obj = poi.ExprBuilder();
  obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[s,g,t] for g in range(nThermal)) + LL_cost*DV.LL[s, t] for t in range(nT));
  Model.add_linear_constraint(DV.theta[s]-obj, poi.Eq, 0);

# on/off status
for g in range(nThermal):
  for t in range(1, nT):
    Model.add_linear_constraint(DV.Ucom[g,t]-DV.Ucom[g,t-1]-DV.Usu[g,t]+DV.Usd[g,t], poi.Eq, 0);

for s in range(nS):
  # balance equation
  for t in range(nT):
    Model.add_linear_constraint(
        poi.quicksum(DV.prod_thermal[s,g,t] for g in range(nThermal))+  # generation from thermal plants
        poi.quicksum(DV.prod_sol[s,g,t] for g in range(nSolar))+        # generation from solar
        poi.quicksum(DV.prod_wind[s,g,t] for g in range(nWind))+        # generation from wind
        DV.store_dis[s,t]-DV.store_ch[s,t]+DV.LL[s,t],                      # discharge-charge + LL
        poi.Eq, demand[s,t]);

  # production capacity for thermal plants
  for g in range(nThermal):
    for t in range(nT):
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-Pmin[g]*DV.Ucom[g,t], poi.Geq,0);

  # ramping
  for g in range(nThermal):
    for t in range(1, nT):
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-DV.prod_thermal[s,g,t-1] -RampU[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[s,g,t-1]-DV.prod_thermal[s,g,t] -RampD[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);

  # availability for VRE
  for t in range(nT):
    for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[s,g,t], poi.Leq, Pmax[g+nThermal]*solar_CF[s,t]);
    for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[s,g,t], poi.Leq, Pmax[g+nThermal+nSolar]*wind_CF[s,t]);

  # storage
  for t in range(1, nT):
    Model.add_linear_constraint(DV.SOC[s,t]-DV.SOC[s,t-1]-store_eff_ch*DV.store_ch[s,t]+DV.store_dis[s,t]/store_eff_dis, poi.Eq, 0);

  Model.add_linear_constraint(DV.SOC[s,0], poi.Eq,0);
  for t in range(nT):
    Model.add_linear_constraint(DV.SOC[s,t], poi.Leq, store_level);
    Model.add_linear_constraint(DV.store_ch[s,t], poi.Leq, store_ch);
    Model.add_linear_constraint(DV.store_dis[s,t], poi.Leq, store_dis);

### SOLVE THE MODEL
Model.set_model_attribute(poi.ModelAttribute.Silent, 0);
Model.optimize();

### POST-PROCESSING
# calculate total generation and load shedding
total_thermal_gen, lost_load = np.zeros(nS), np.zeros(nS);
total_VRE_gen = np.zeros(nS);
for s in range(nS):
  for t in range(nT):
      for g in range(nThermal): total_thermal_gen[s] += Model.get_value(DV.prod_thermal[s,g,t]);
      for g in range(nSolar): total_VRE_gen[s] += Model.get_value(DV.prod_sol[s,g,t]);
      for g in range(nWind): total_VRE_gen[s] += Model.get_value(DV.prod_wind[s,g,t]);
      lost_load[s] += Model.get_value(DV.LL[s,t]);
scenario_obj = np.zeros(nS);
for s in range(nS):
  scenario_obj[s] = round(Model.get_value(DV.theta[s]));
  print(f'scenario {s}, prob: {np.round(probs[s],2)},  obj = {scenario_obj[s]}, Thermal gen: {round(total_thermal_gen[s])} MWh,  VRE gen: {round(total_VRE_gen[s])} MWh,\
   lost load: {round(lost_load[s])} MWh,  total load: {round(demand[s].sum())} MWh');
print(f'\n Total Objective value: {np.round(Model.get_model_attribute(poi.ModelAttribute.ObjectiveValue),1)}, \
first-stage obj: {round(Model.get_value(DV.s1_obj))}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');

UC_obj_SP = Model.get_model_attribute(poi.ModelAttribute.ObjectiveValue);

scenario 0, prob: 0.01,  obj = 581202.0, Thermal gen: 199998 MWh,  VRE gen: 158597 MWh,   lost load: 0 MWh,  total load: 358793 MWh
scenario 1, prob: 0.5,  obj = 1154318.0, Thermal gen: 322648 MWh,  VRE gen: 169583 MWh,   lost load: 0 MWh,  total load: 492422 MWh
scenario 2, prob: 0.14,  obj = 394584.0, Thermal gen: 156780 MWh,  VRE gen: 189329 MWh,   lost load: 0 MWh,  total load: 346272 MWh
scenario 3, prob: 0.2,  obj = 787186.0, Thermal gen: 250739 MWh,  VRE gen: 172629 MWh,   lost load: 0 MWh,  total load: 423425 MWh
scenario 4, prob: 0.16,  obj = 1154318.0, Thermal gen: 322648 MWh,  VRE gen: 169583 MWh,   lost load: 0 MWh,  total load: 492422 MWh

 Total Objective value: 1132502.3, first-stage obj: 162186, 	 Elapsed time: 4.58 (s)


<br><br>

## Robust UC

**<font color='blue'>Mathematical Model </font>**   
    \begin{align}
    \min & \quad \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}}(c^{\text{su}}_g u^{\text{su}}_{gt}+c^{\text{sd}}_g u^{\text{sd}}_{gt})+ \Theta\\
    \text{s.t.} \ &\Theta\geq \sum_{g\in \mathcal{H}} \sum_{t\in \mathcal{T}} c_g p_{sgt} + \sum_{t\in \mathcal{T}}c^{\text{VoLL}} a_{st}& s\in \mathcal{S}\\
    & u^{\text{gen}}_{gt} = u^{\text{gen}}_{g,t-1}+u^{\text{su}}_{gt}-u^{\text{sd}}_{gt}& g\in \mathcal{G}, t\in \mathcal{T}\\
    & \sum_{g\in \mathcal{G}} p_{sgt} + s^{\text{dis}}_{st} - s^{\text{ch}}_{st}+a_{st} = D_{st} & s\in \mathcal{S}, t\in \mathcal{T}\\
    & P^{\text{min}}_g u^{\text{gen}}_{gt} \leq p_{sgt} \leq P^{\text{max}}_g u^{\text{gen}}_{gt} & s\in \mathcal{S}, g\in \mathcal{G}, t\in \mathcal{T}\\
    & p_{sgt}-p_{sg,t-1} \leq R^{\text{up}}_g P^{\text{max}}_gu^{\text{gen}}_{gt}, \qquad  p_{sg,t-1}-p_{sgt} \leq R^{\text{dn}}_g P^{\text{max}}_g u^{\text{gen}}_{gt} & s\in \mathcal{S}, g\in \mathcal{H}, t\in \mathcal{T}\backslash t_0\\
    & p_{sgt}\leq \rho_{sgt}P^{\text{max}}_g & s\in \mathcal{S}, g\in \mathcal{V}, t\in \mathcal{T}\\
    &s^{\text{soc}}_{st} = s^{\text{soc}}_{s,t-1}+ \gamma^{\text{ch}} s^{\text{ch}}_{st} - \frac{s^{\text{dis}}_{st}}{\gamma^{\text{dis}}}, \qquad s^{\text{soc}}_{s0}=0&s\in \mathcal{S}, t\in \mathcal{T}\backslash t_0\\
    &s^{\text{soc}}_{st}\leq y^{\text{lev}}, \quad s^{\text{ch}}_{st}\leq y^{\text{ch}} ,\quad s^{\text{ch}}_{st}\leq  y^{\text{dis}}&s\in \mathcal{S}, t\in \mathcal{T}
\end{align}


In [14]:
np.random.seed(2025);
time_stamp = time.time();
### SETS
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
# demand is the demand matrix, solar_CF and wind_CF are the solar and wind capacity factors
gen_cost = np.random.randint(low=2, high=7, size=nThermal); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nThermal);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nThermal);
store_level = 0.2*demand.max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;
commit_cost = np.random.randint(500, 1500, size=nThermal);
startup_cost = np.random.randint(1000, 3000, size=nThermal);
shutdown_cost = np.random.randint(200, 2000, size=nThermal);
probs = np.random.dirichlet(np.ones(nS), size=1)[0];

# MODEL INSTANCE
Model = highs.Model();

### DECISION VARIABLES
# define a class to distinguish the variables from parameters
class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[]; Ucom=[]; Usu=[]; Usd=[]; theta=[];s1_obj=[]; total_obj=[]

DV.prod_thermal = Model.add_variables(range(nS), range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.prod_sol = Model.add_variables(range(nS), range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.prod_wind = Model.add_variables(range(nS), range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.LL = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
DV.SOC = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
DV.store_ch = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
DV.store_dis = Model.add_variables(range(nS), range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
DV.Ucom = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # generator on/off status
DV.Usu = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # startup decision
DV.Usd = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # shutdown decision
DV.theta = Model.add_variables(range(nS), domain=poi.VariableDomain.Continuous); # objective function for second-stage problmes, free variable
DV.s1_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for first-stage problme, free variable
DV.Theta = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for first-stage problme, free variable
DV.total_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # total objective function, free variable

### OBJECTIVE FUNCTION
obj_total = poi.ExprBuilder();
# first-stage objective
obj_total += poi.quicksum(poi.quicksum(commit_cost[g]*DV.Ucom[g,t]+ startup_cost[g]*DV.Usu[g,t]+shutdown_cost[g]*DV.Usd[g,t] for g in range(nThermal)) for t in range(nT));
Model.add_linear_constraint(DV.s1_obj-obj_total, poi.Eq, 0);
obj_total += DV.Theta;
Model.add_linear_constraint(DV.total_obj-obj_total, poi.Eq, 0);
Model.set_objective(obj_total, poi.ObjectiveSense.Minimize)

### CONSTRAINTS
# cost components
for s in range(nS):
  obj = poi.ExprBuilder();
  obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[s,g,t] for g in range(nThermal)) + LL_cost*DV.LL[s, t] for t in range(nT));
  Model.add_linear_constraint(DV.theta[s]-obj, poi.Eq, 0);
  Model.add_linear_constraint(DV.Theta - DV.theta[s], poi.Geq, 0)

# on/off status
for g in range(nThermal):
  for t in range(1, nT):
    Model.add_linear_constraint(DV.Ucom[g,t]-DV.Ucom[g,t-1]-DV.Usu[g,t]+DV.Usd[g,t], poi.Eq, 0);

for s in range(nS):
  # balance equation
  for t in range(nT):
    Model.add_linear_constraint(
        poi.quicksum(DV.prod_thermal[s,g,t] for g in range(nThermal))+  # generation from thermal plants
        poi.quicksum(DV.prod_sol[s,g,t] for g in range(nSolar))+        # generation from solar
        poi.quicksum(DV.prod_wind[s,g,t] for g in range(nWind))+        # generation from wind
        DV.store_dis[s,t]-DV.store_ch[s,t]+DV.LL[s,t],                      # discharge-charge + LL
        poi.Eq, demand[s,t]);

  # production capacity for thermal plants
  for g in range(nThermal):
    for t in range(nT):
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-Pmin[g]*DV.Ucom[g,t], poi.Geq,0);

  # ramping
  for g in range(nThermal):
    for t in range(1, nT):
      Model.add_linear_constraint(DV.prod_thermal[s,g,t]-DV.prod_thermal[s,g,t-1] -RampU[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[s,g,t-1]-DV.prod_thermal[s,g,t] -RampD[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);

  # availability for VRE
  for t in range(nT):
    for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[s,g,t], poi.Leq, Pmax[g+nThermal]*solar_CF[s,t]);
    for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[s,g,t], poi.Leq, Pmax[g+nThermal+nSolar]*wind_CF[s,t]);

  # storage
  for t in range(1, nT):
    Model.add_linear_constraint(DV.SOC[s,t]-DV.SOC[s,t-1]-store_eff_ch*DV.store_ch[s,t]+DV.store_dis[s,t]/store_eff_dis, poi.Eq, 0);

  Model.add_linear_constraint(DV.SOC[s,0], poi.Eq,0);
  for t in range(nT):
    Model.add_linear_constraint(DV.SOC[s,t], poi.Leq, store_level);
    Model.add_linear_constraint(DV.store_ch[s,t], poi.Leq, store_ch);
    Model.add_linear_constraint(DV.store_dis[s,t], poi.Leq, store_dis);

### SOLVE THE MODEL
Model.set_model_attribute(poi.ModelAttribute.Silent, 0);
Model.optimize();

### POST-PROCESSING
# calculate total generation and load shedding
total_thermal_gen, lost_load = np.zeros(nS), np.zeros(nS);
total_VRE_gen = np.zeros(nS);
for s in range(nS):
  for t in range(nT):
      for g in range(nThermal): total_thermal_gen[s] += Model.get_value(DV.prod_thermal[s,g,t]);
      for g in range(nSolar): total_VRE_gen[s] += Model.get_value(DV.prod_sol[s,g,t]);
      for g in range(nWind): total_VRE_gen[s] += Model.get_value(DV.prod_wind[s,g,t]);
      lost_load[s] += Model.get_value(DV.LL[s,t]);
scenario_obj = np.zeros(nS);
for s in range(nS):
  scenario_obj[s] = round(Model.get_value(DV.theta[s]));
  print(f'scenario {s},  obj = {scenario_obj[s]}, Thermal gen: {round(total_thermal_gen[s])} MWh,  VRE gen: {round(total_VRE_gen[s])} MWh,\
   lost load: {round(lost_load[s])} MWh,  total load: {round(demand[s].sum())} MWh');
print(f'\n Total Objective value: {np.round(Model.get_model_attribute(poi.ModelAttribute.ObjectiveValue),1)}, \
first-stage obj: {round(Model.get_value(DV.s1_obj))}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');
UC_obj_Robust = Model.get_model_attribute(poi.ModelAttribute.ObjectiveValue);

scenario 0,  obj = 1154318.0, Thermal gen: 323008 MWh,  VRE gen: 35659 MWh,   lost load: 0 MWh,  total load: 358793 MWh
scenario 1,  obj = 1154318.0, Thermal gen: 322648 MWh,  VRE gen: 169583 MWh,   lost load: 0 MWh,  total load: 492422 MWh
scenario 2,  obj = 1154318.0, Thermal gen: 322746 MWh,  VRE gen: 22743 MWh,   lost load: 0 MWh,  total load: 346272 MWh
scenario 3,  obj = 1154318.0, Thermal gen: 322464 MWh,  VRE gen: 100505 MWh,   lost load: 0 MWh,  total load: 423425 MWh
scenario 4,  obj = 1154318.0, Thermal gen: 322648 MWh,  VRE gen: 169583 MWh,   lost load: 0 MWh,  total load: 492422 MWh

 Total Objective value: 1316504.3, first-stage obj: 162186, 	 Elapsed time: 7.12 (s)




# **<font color='blue'>5. Value of Stochastic Solution (VSS) </font>**  <a name="VSS"></a>

There are different ways to assess whether it is worth characterizing the uncertainty via a stochastic programming approach and VSS is one of the common methods. In essence, it measures the benefit of explicitly incorporating uncertainty into a model over using average values for uncertain parameters. Let $Z_{SP}$ be the solution for the stochastic problem. The following steps are taken to calculate VSS:
- Solve Expected Value (EV) problem in which the uncertain parameters are replaced by their expectation over scenarios. Store the values of the first stage variables.
- Solve Expected result of using EV decisions (EEV), which is the same as the stochastic problem except the first-stage variables are set to their values in the previous step.

Then the VSS is calculated as $EEV-Z_{SP}$ or in it normalized form as $\frac{100\times(EEV-Z_{SP})}{EEV}$.




**Step 1:** Expected value problem (EV)


In [15]:
np.random.seed(2025);
time_stamp = time.time();
### SETS
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
# demand is the demand matrix, solar_CF and wind_CF are the solar and wind capacity factors
gen_cost = np.random.randint(low=2, high=7, size=nThermal); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nThermal);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nThermal);
store_level = 0.2*demand.max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;
commit_cost = np.random.randint(500, 1500, size=nThermal);
startup_cost = np.random.randint(1000, 3000, size=nThermal);
shutdown_cost = np.random.randint(200, 2000, size=nThermal);
probs = np.random.dirichlet(np.ones(nS), size=1)[0];

# MODEL INSTANCE
Model = highs.Model();


### DECISION VARIABLES
# define a class to distinguish the variables from parameters
class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[]; Ucom=[]; Usu=[]; Usd=[];s1_obj=[]; s2_obj=[]; total_obj=[]

DV.prod_thermal = Model.add_variables( range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.prod_sol = Model.add_variables(range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.prod_wind = Model.add_variables(range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
DV.LL = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
DV.SOC = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
DV.store_ch = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
DV.store_dis = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
DV.Ucom = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # generator on/off status
DV.Usu = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # startup decision
DV.Usd = Model.add_variables(range(nThermal), range(nT), domain=poi.VariableDomain.Binary); # shutdown decision
DV.s1_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for first-stage problme, free variable
DV.s2_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for second-stage problme, free variable
DV.total_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # total objective function, free variable

### OBJECTIVE FUNCTION
obj_total = poi.ExprBuilder();
# first-stage objective
obj_total += poi.quicksum(poi.quicksum(commit_cost[g]*DV.Ucom[g,t]+ startup_cost[g]*DV.Usu[g,t]+shutdown_cost[g]*DV.Usd[g,t] for g in range(nThermal)) for t in range(nT));
Model.add_linear_constraint(DV.s1_obj-obj_total, poi.Eq, 0);
# second-stage objective
obj_total += DV.s2_obj;
Model.add_linear_constraint(DV.total_obj-obj_total, poi.Eq, 0);
Model.set_objective(obj_total, poi.ObjectiveSense.Minimize)

### CONSTRAINTS
# cost components

obj = poi.ExprBuilder();
obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[g,t] for g in range(nThermal)) + LL_cost*DV.LL[t] for t in range(nT));
Model.add_linear_constraint(DV.s2_obj-obj, poi.Eq, 0);

# on/off status
for g in range(nThermal):
  for t in range(1, nT):
    Model.add_linear_constraint(DV.Ucom[g,t]-DV.Ucom[g,t-1]-DV.Usu[g,t]+DV.Usd[g,t], poi.Eq, 0);

# balance equation
for t in range(nT):
  Model.add_linear_constraint(
      poi.quicksum(DV.prod_thermal[g,t] for g in range(nThermal))+  # generation from thermal plants
      poi.quicksum(DV.prod_sol[g,t] for g in range(nSolar))+        # generation from solar
      poi.quicksum(DV.prod_wind[g,t] for g in range(nWind))+        # generation from wind
      DV.store_dis[t]-DV.store_ch[t]+DV.LL[t],                      # discharge-charge + LL
      poi.Eq, probs@demand[:,t]);

# production capacity for thermal plants
for g in range(nThermal):
  for t in range(nT):
    Model.add_linear_constraint(DV.prod_thermal[g,t]-Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
    Model.add_linear_constraint(DV.prod_thermal[g,t]-Pmin[g]*DV.Ucom[g,t], poi.Geq,0);

# ramping
for g in range(nThermal):
  for t in range(1, nT):
    Model.add_linear_constraint(DV.prod_thermal[g,t]-DV.prod_thermal[g,t-1] -RampU[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);
    Model.add_linear_constraint(DV.prod_thermal[g,t-1]-DV.prod_thermal[g,t] -RampD[g]*Pmax[g]*DV.Ucom[g,t], poi.Leq, 0);

# availability for VRE
for t in range(nT):
  for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[g,t], poi.Leq, Pmax[g+nThermal]*(probs@solar_CF[:,t]));
  for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[g,t], poi.Leq, Pmax[g+nThermal+nSolar]*(probs@wind_CF[:,t]));

# storage
for t in range(1, nT):
  Model.add_linear_constraint(DV.SOC[t]-DV.SOC[t-1]-store_eff_ch*DV.store_ch[t]+DV.store_dis[t]/store_eff_dis, poi.Eq, 0);

Model.add_linear_constraint(DV.SOC[0], poi.Eq,0);
for t in range(nT):
  Model.add_linear_constraint(DV.SOC[t], poi.Leq, store_level);
  Model.add_linear_constraint(DV.store_ch[t], poi.Leq, store_ch);
  Model.add_linear_constraint(DV.store_dis[t], poi.Leq, store_dis);

### SOLVE THE MODEL
Model.set_model_attribute(poi.ModelAttribute.Silent, 0);
Model.optimize();

### POST-PROCESSING
print(f'\n Total Objective value: {np.round(Model.get_model_attribute(poi.ModelAttribute.ObjectiveValue),1)}, \
first-stage obj: {round(Model.get_value(DV.s1_obj))}, \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');



 Total Objective value: 1097253.1, first-stage obj: 147407, 	 Elapsed time: 2.77 (s)


Get the UC decisions to be used in the EEV problem

In [16]:
Ucom_val, Usu_val, Usd_val = np.zeros((nThermal, nT)), np.zeros((nThermal, nT)), np.zeros((nThermal, nT));
s1_obj = Model.get_value(DV.s1_obj);
for g in range(nThermal):
  for t in range(nT):
    Ucom_val[g,t] = Model.get_value(DV.Ucom[g,t]);
    Usu_val[g,t] = Model.get_value(DV.Usu[g,t]);
    Usd_val[g,t] = Model.get_value(DV.Usd[g,t]);

**Step 2:** Solve the second-stage problem for each scenario, given the UC decisions

In [17]:
s2_obj = np.zeros(nS);
np.random.seed(2025);
### SETS
nThermal = 10;
nSolar = 5;
nWind = 5;
nT = len(demand[0]);

### PARAMETERS
# demand is the demand matrix, solar_CF and wind_CF are the solar and wind capacity factors
gen_cost = np.random.randint(low=2, high=7, size=nThermal); # marginal generation cost (variable )
LL_cost = 1e4;  # penalty for lost load
Pmax =  np.random.randint(int(0.2*demand.max()/nThermal), int(demand.max()/nThermal), size=nThermal+nSolar+nWind);
Pmin = np.zeros(nThermal+nSolar+nWind);
RampU =  np.random.uniform(low=0.2, high=0.7, size=nThermal);
RampD =  np.random.uniform(low=0.1, high=0.5, size=nThermal);
store_level = 0.2*demand.max();
store_ch, store_dis = store_level/4, store_level/4;
store_eff_ch, store_eff_dis = 0.9, 0.9;
commit_cost = np.random.randint(500, 1500, size=nThermal);
startup_cost = np.random.randint(1000, 3000, size=nThermal);
shutdown_cost = np.random.randint(200, 2000, size=nThermal);
probs = np.random.dirichlet(np.ones(nS), size=1)[0];

for s in range(nS):
  time_stamp = time.time();
  ### Sets and parameters as before

  # MODEL INSTANCE
  Model = highs.Model();


  ### DECISION VARIABLES
  # define a class to distinguish the variables from parameters
  class DV(): prod_thermal=[]; prod_vre=[]; LL=[]; SOC=[]; store_ch=[]; store_dis=[]; s2_obj=[];

  DV.prod_thermal = Model.add_variables( range(nThermal), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
  DV.prod_sol = Model.add_variables(range(nSolar), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
  DV.prod_wind = Model.add_variables(range(nWind), range(nT), lb = 0,domain=poi.VariableDomain.Continuous);
  DV.LL = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # lost load (shedding), continuous positive variable
  DV.SOC = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage state of charge, continuous positive variable
  DV.store_ch = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage charge, continuous positive variable
  DV.store_dis = Model.add_variables(range(nT), lb = 0, domain=poi.VariableDomain.Continuous); # storage discharge, continuous positive variable
  DV.s2_obj = Model.add_variable(domain=poi.VariableDomain.Continuous); # objective function for second-stage problme, free variable

  ### OBJECTIVE FUNCTION
  # second-stage objective
  obj_total = poi.ExprBuilder();
  obj_total += DV.s2_obj;
  Model.set_objective(obj_total, poi.ObjectiveSense.Minimize)

  ### CONSTRAINTS
  # cost components
  obj = poi.ExprBuilder();
  obj += poi.quicksum(poi.quicksum(gen_cost[g]*DV.prod_thermal[g,t] for g in range(nThermal)) + LL_cost*DV.LL[t] for t in range(nT));
  Model.add_linear_constraint(DV.s2_obj-obj, poi.Eq, 0);

  # # on/off status
  # for g in range(nThermal):
  #   for t in range(1, nT):
  #     Model.add_linear_constraint(DV.Ucom[g,t]-DV.Ucom[g,t-1]-DV.Usu[g,t]+DV.Usd[g,t], poi.Eq, 0);

  # balance equation
  for t in range(nT):
    Model.add_linear_constraint(
        poi.quicksum(DV.prod_thermal[g,t] for g in range(nThermal))+  # generation from thermal plants
        poi.quicksum(DV.prod_sol[g,t] for g in range(nSolar))+        # generation from solar
        poi.quicksum(DV.prod_wind[g,t] for g in range(nWind))+        # generation from wind
        DV.store_dis[t]-DV.store_ch[t]+DV.LL[t],                      # discharge-charge + LL
        poi.Eq, demand[s,t]);

  # production capacity for thermal plants
  for g in range(nThermal):
    for t in range(nT):
      Model.add_linear_constraint(DV.prod_thermal[g,t]-Pmax[g]*Ucom_val[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[g,t]-Pmin[g]*Ucom_val[g,t], poi.Geq,0);

  # ramping
  for g in range(nThermal):
    for t in range(1, nT):
      Model.add_linear_constraint(DV.prod_thermal[g,t]-DV.prod_thermal[g,t-1] -RampU[g]*Pmax[g]*Ucom_val[g,t], poi.Leq, 0);
      Model.add_linear_constraint(DV.prod_thermal[g,t-1]-DV.prod_thermal[g,t] -RampD[g]*Pmax[g]*Ucom_val[g,t], poi.Leq, 0);

  # availability for VRE
  for t in range(nT):
    for g in range(nSolar):Model.add_linear_constraint(DV.prod_sol[g,t], poi.Leq, Pmax[g+nThermal]*solar_CF[s,t]);
    for g in range(nWind):Model.add_linear_constraint(DV.prod_wind[g,t], poi.Leq, Pmax[g+nSolar]*wind_CF[s,t]);

  # storage
  for t in range(1, nT):
    Model.add_linear_constraint(DV.SOC[t]-DV.SOC[t-1]-store_eff_ch*DV.store_ch[t]+DV.store_dis[t]/store_eff_dis, poi.Eq, 0);

  Model.add_linear_constraint(DV.SOC[0], poi.Eq,0);
  for t in range(nT):
    Model.add_linear_constraint(DV.SOC[t], poi.Leq, store_level);
    Model.add_linear_constraint(DV.store_ch[t], poi.Leq, store_ch);
    Model.add_linear_constraint(DV.store_dis[t], poi.Leq, store_dis);

  ### SOLVE THE MODEL
  Model.set_model_attribute(poi.ModelAttribute.Silent, 0);
  Model.optimize();

  ### POST-PROCESSING
  ll=0;
  for t in range(nT):
    ll += Model.get_value(DV.LL[t]);
  print(f' Scenario: {s}, second-stage obj: {round(Model.get_value(DV.s2_obj))},\
   lost load: {round(ll)} ({round(100*ll/demand[s].sum())}%), \t Elapsed time: {np.round(time.time()-time_stamp, 2)} (s)');
  s2_obj[s] = probs[s]*Model.get_value(DV.s2_obj);

EEV = s1_obj+sum(s2_obj);

 Scenario: 0, second-stage obj: 493857,   lost load: 0 (0%), 	 Elapsed time: 0.03 (s)
 Scenario: 1, second-stage obj: 121602696,   lost load: 12066 (2%), 	 Elapsed time: 0.03 (s)
 Scenario: 2, second-stage obj: 289639,   lost load: 0 (0%), 	 Elapsed time: 0.03 (s)
 Scenario: 3, second-stage obj: 657594,   lost load: 0 (0%), 	 Elapsed time: 0.03 (s)
 Scenario: 4, second-stage obj: 121602696,   lost load: 12066 (2%), 	 Elapsed time: 0.02 (s)


Calculate the VSS:

In [18]:
100*(EEV-UC_obj_SP)/EEV

np.float64(98.58042677245034)

\Comparison between the objective functions obtained by different approaches:

In [20]:
print(f'Best case deterministic scenario: {round(UC_obj_det.min())}');
print(f'Worst case deterministic scenario: {round(UC_obj_det.max())}');
print(f'Stochastic programming outcome: {round(UC_obj_SP)}');
print(f'Robust optimization outcome: {round(UC_obj_Robust)}');
print(f'Expected value problem outcome: {round(EEV)}');

Best case deterministic scenario: 500310
Worst case deterministic scenario: 1316504
Stochastic programming outcome: 1132502
Robust optimization outcome: 1316504
Expected value problem outcome: 79777656


<br><br><br><br>


# **<font color='blue'>6. Case Study Questions </font>** <a name="questions"></a>

**<font color='red'> Questions </font>**
1. Consider the 100 extreme days (scenarios) generated as part of the case study 2 assignment. Select 30 possible realizations randomly and divide the set into 20 training and 10 test scenarios. For the UC models developed in this recitation:
- run the deterministic UC on each of the training scenarios and record the first-stage decisions that resulted in the highest objective value.
- run the stochastic UC and record its first-stage variables.
- run the robust UC and record its first-stage variables.

Now evaluate each of the recorded first-stage variable sets on the 10 test scenarios, just as we did in step 2 of the VSS calculation (dispatch problem with commitment decisions are given). Record the average out-of-sample performance for each of the paradigms. What do you observe? Which approach has a better performance and why? Comment on the total average cost (first stage + average out-of-sample cost) for each approach.

<br>
<br>
<br>



# **<font color='blue'>7. References and Further Reading </font>** <a name="references"></a>


- Conejo, A. J., & Baringo, L. (2018). Power system operations (Vol. 11). New York: Springer. ([link](https://link.springer.com/book/10.1007/978-3-319-69407-8)) - [section 7 covers ED and UC]
- Sun, X. A., & Conejo, A. J. (2021). Robust optimization in electric energy systems (Vol. 313). Cham: Springer.([link](https://link.springer.com/book/10.1007/978-3-030-85128-6))
- Ross, S. M. (2014). Introduction to stochastic dynamic programming. Academic press.([link](https://link.springer.com/book/10.1007/978-1-4614-0237-4))
- Xavier, Á. S., Qiu, F., & Ahmed, S. (2021). Learning to solve large-scale security-constrained unit commitment problems. INFORMS Journal on Computing, 33(2), 739-756. ([link](https://pubsonline.informs.org/doi/abs/10.1287/ijoc.2020.0976))
- Chen, W., Park, S., Tanneau, M., & Van Hentenryck, P. (2022). Learning optimization proxies for large-scale security-constrained economic dispatch. Electric Power Systems Research, 213, 108566. ([link](https://www.sciencedirect.com/science/article/pii/S0378779622006629?casa_token=UHByIil0de4AAAAA:iuAn6VKvqTb9Ulev1O8Mc5SjxKnhY8wqCcV7z5wxTckWhyvb5-LUw-xw8rDNjWy6YIm3hbt3))

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

